# Baseline script for SomaNews Clustering
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

## Load Data
Load Data from database

In [2]:
train = pd.read_pickle("../datastore/weekly.p")
train.head()

,author,category,content,description,imageURL,link,provider,providerNewsID,publishedAt,title
publishedAt,,,,,,,,,,
2016-09-04 23:28:00,김재철 숭실대 전기공학부 교수(부총장),경제,모든 사물이 인터넷으로 연결된다는 사물인터넷(IoT·Internet of Thing...,"[ESS산업, 현재와 미래]\n\n- 배터리로 움직이는 'BoT 시대'\nESS 발...",http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090500409,2016-09-04 23:28:00,電氣 저장했다 꺼내 쓰는 ESS… 4년 뒤 150억달러 시장 선다
2016-09-04 23:24:00,이위재 기자,nativeAD > 기업뉴스,지난달 말 산업통상자원부는 ESS 관련 산업 도약을 위해 민·관·학계가 함께하는 '...,"산업부, 신산업 규제 대폭 손질… 할인 기간 늘려 기업 투자 활성\n'ESS 융합 ...",http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090501185,2016-09-04 23:24:00,ESS 할인 요금제 기간 10년으로
2016-09-04 23:20:00,김승범 기자,경제,정유업계에 긴장감이 돌고 있다. 올 상반기에는 세계 경기 불황 속에서도 사상 최고 ...,[8월 정제마진 배럴당 3.9달러 … '4~5달러' 손익분기점 밑돌아]\n\n- 공...,http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090500403,2016-09-04 23:20:00,중국發 공급과잉·低유가 먹구름… 잘나가던 정유업계 긴장
2016-09-04 22:54:00,베를린=심현정 기자,경제,지난 3일(현지 시각) 오후 유럽 최대 가전박람회 'IFA 2016'이 열리고 있는...,독일 IFA 전시회\n\n- 中 유비텍 '지무로봇'\n레고 블록에 모터 등 연결해 ...,http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090500405,2016-09-04 22:54:00,레고 로봇·스마트 캐디·게임 칫솔… 獨서 빛나는 작은 기업들
2016-09-04 22:52:00,베를린=심현정 기자,nativeAD > LG전자,"""기술력에 대한 자신감이 없다면 우리 가전제품의 핵심 부품을 다른 경쟁 업체에 판매...",[독일 IFA 전시회]\n\n조성진 LG전자 가전담당 사장,http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090501158,2016-09-04 22:52:00,"""경쟁社에 핵심부품 파는 건, 기술력에 자신감 있기 때문"""


## Preprocessing
0. Datetime (16-09-11 ~ 16-09-17)
1. Remove stopwords (regex, hanja)
2. POS Tagging with KoNLPy, Mecab

In [3]:
import datetime
from konlpy.tag import Mecab
import hanja
import re

In [4]:
mecab = Mecab()
print(mecab.pos(u'네, 안녕하세요 나는 정답'))

[('네', 'IC'), (',', 'SC'), ('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EP+EF'), ('나', 'NP'), ('는', 'JX'), ('정답', 'NNG')]


In [5]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', '', text)
    text = text.replace(u"카드뉴스", '').replace(u"조선일보", '')
    return text

### POS Tagging
일반명사, 고유명사, 외국어 (NNG, NNP, SL)

In [6]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]

In [7]:
train['title'] = train['title'].apply(lambda text: text_cleaning(text))
title = [tokenize(each[1]['title']) for each in train.iterrows()]

## Training
1. Feature extraction - TfidVectorizer
2. Decomposition - PCA
3. Cluster - KMeans

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [9]:
vectorizer = TfidfVectorizer(lowercase=False)
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)

In [11]:
x_list_10d = PCA(n_components=100).fit_transform(x_list.toarray())
x_list_10d.shape

(3068, 100)

### Scoring

In [12]:
from sklearn.metrics import silhouette_samples, silhouette_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Best Silhoutte Score

In [13]:
best_score = 0.0
best_k = 0

In [14]:
for k in range(10, 25):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list_10d)
    score = silhouette_score(x_list, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    print("In Clusters =", k, ", Score is : %0.3f" % score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In Clusters = 10 , Score is : 0.012
In Clusters = 11 , Score is : 0.013
In Clusters = 12 , Score is : 0.013
In Clusters = 13 , Score is : 0.016
In Clusters = 14 , Score is : 0.015
In Clusters = 15 , Score is : 0.015
In Clusters = 16 , Score is : 0.017
In Clusters = 17 , Score is : 0.018
In Clusters = 18 , Score is : 0.016
In Clusters = 19 , Score is : 0.020
In Clusters = 20 , Score is : 0.018
In Clusters = 21 , Score is : 0.018
In Clusters = 22 , Score is : 0.023
In Clusters = 23 , Score is : 0.021
In Clusters = 24 , Score is : 0.020
In Clusters = 22 , Best score is : 0.023


### K-Means Algorithm

In [15]:
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list)
labels = km.labels_
centroids = km.cluster_centers_
print(km.inertia_)

2898.2128598488016


In [16]:
train = train.reset_index(drop=True)
x_list_vector = x_list_10d.tolist()
train['cluster'] = labels
train['vector'] = x_list_vector
train.head(2)

,author,category,content,description,imageURL,link,provider,providerNewsID,publishedAt,title,cluster,vector
0,김재철 숭실대 전기공학부 교수(부총장),경제,모든 사물이 인터넷으로 연결된다는 사물인터넷(IoT·Internet of Thing...,"[ESS산업, 현재와 미래]\n\n- 배터리로 움직이는 'BoT 시대'\nESS 발...",http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090500409,2016-09-04 23:28:00,전기 저장했다 꺼내 쓰는 ESS 4년 뒤 150억달러 시장 선다,19,"[-0.013763079439635596, -0.015551843547112382,..."
1,이위재 기자,nativeAD > 기업뉴스,지난달 말 산업통상자원부는 ESS 관련 산업 도약을 위해 민·관·학계가 함께하는 '...,"산업부, 신산업 규제 대폭 손질… 할인 기간 늘려 기업 투자 활성\n'ESS 융합 ...",http://image.chosun.com/sitedata/image/201609/...,http://news.chosun.com/site/data/html_dir/2016...,chosun,2016090501185,2016-09-04 23:24:00,ESS 할인 요금제 기간 10년으로,0,"[-0.010411064433675359, -0.010700426667365417,..."


## Choose Best Cluster
1. Cluster size < 500
2. Recent published
3. Minimum inertia

### Compare best cluster

In [62]:
sample_silhouette_values = silhouette_samples(x_list_10d, labels)
sample_silhouette_score = []
best_cluster = []
cluster_num = best_k

for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    print('Cluster %d: ' % (i + 1), abs(ith_cluster_silhouette_values.mean()))
    sample_silhouette_score.append(abs(ith_cluster_silhouette_values.mean()))

sample_silhouette_score.sort()
sample_silhouette_score = sample_silhouette_score[:8]

Cluster 1:  0.00961884620919
Cluster 2:  0.0212616670536
Cluster 3:  0.272663224706
Cluster 4:  0.228436269399
Cluster 5:  0.0493090627147
Cluster 6:  0.282643845803
Cluster 7:  0.0534744345398
Cluster 8:  0.230344292583
Cluster 9:  0.103958259459
Cluster 10:  0.016690414179
Cluster 11:  0.121631960119
Cluster 12:  0.141472186759
Cluster 13:  0.133416583816
Cluster 14:  0.190304604249
Cluster 15:  0.309731105291
Cluster 16:  0.0187654905993
Cluster 17:  0.103949137684
Cluster 18:  0.366946161523
Cluster 19:  0.18713714927
Cluster 20:  0.0150458296667
Cluster 21:  0.0124724787646
Cluster 22:  0.331535249665


In [63]:
for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    if abs(ith_cluster_silhouette_values.mean()) in sample_silhouette_score:
        best_cluster.append(i+1)
        
best_cluster

[1, 2, 5, 7, 10, 16, 20, 21]

In [67]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(train[train['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i + 1), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 1: Size 28


,title,category
1830,사설압수수색 받는 민정수석이 현직에 있는 게 정상인가,뉴스 > 정치 > 정치일반
2149,전직 경찰 총경 경찰 멱살 잡은 한선교 의원 고발할 것,정치 > 국회·정당
2352,박 대통령 러시아중국미국 연쇄 정상회담,정치 > 청와대
773,온라인 당원 위세에 친문의원들도 꼼짝못하네,정치 > 국회ㆍ정당
2128,박 대통령 5일 오전 시진핑 주석과 한중 정상회담,정치 > 청와대
2511,포토못배운 표난다는 의원님 배운 티는 이런건가요,정치 > 국회·정당
883,친문 독식 이끈 온라인 파워에 말조심하는 의원들,정치 > 국회ㆍ정당
2415,청주대 학생들 학교 정상화 위한 언론 독립선언,사회 > 전국
2517,민병두 의원 정의기억재단에 1천만원 기부,정치 > 국회·정당
2577,9월45일 항저우 G20 정상회의 때 한중 정상회담 열릴 듯,정치 > 외교





Cluster 2: Size 49


,title,category
2016,사회공헌 Together우리다문화장학재단 출범 총 2380명에 장학금,뉴스 > 경제 > 기업·CEO
2788,아기 울음소리 들리는 사회를 위해 신혼부부 지원 패키지 마련,경제 > 경제일반
1993,사회공헌 Together매년 영업이익 3 이상 사회공헌사업에 지출,뉴스 > 경제 > 기업·CEO
2001,사회공헌 Together푸르메재단 넥슨어린이재활병원에 200억 기부,뉴스 > 경제 > 기업·CEO
1995,사회공헌 Together출산 장려아동학대 근절에 온힘,뉴스 > 경제 > 기업·CEO
2018,사회공헌 Together기업 CEO단체장 명예이장 위촉해 마을에 활력,뉴스 > 경제 > 기업·CEO
1992,사회공헌 Together국내 산지 농산물 활용 농가와의 상생 힘써,뉴스 > 경제 > 기업·CEO
2002,사회공헌 Together임직원 교육 기부 미래 소프트웨어 인재 육성,뉴스 > 경제 > 기업·CEO
2014,사회공헌 Together서울 중구청에 소외 계층 생계비 1억 원 전달,뉴스 > 경제 > 기업·CEO
2000,사회공헌 Together국립묘지와 1사1묘역 자매결연 맺어 나라사랑,뉴스 > 경제 > 기업·CEO





Cluster 3: Size 49


...

,title,category
143,만물상 C형 간염,사설ㆍ칼럼 > 내부칼럼 > 만물상
1425,사설세 번째 콜레라 발생 보건당국 초기대응 또 뚫렸나,뉴스 > 사회 > 보건·복지
1033,이번엔 대학병원서 C형 간염 감염 사태,뉴스 > 사회 > 사회일반
2385,종합병원인 건국대 충주병원에서도 C형 간염 3명 발생,사회 > 의료·건강
2423,거제 콜레라 환자 3명은 같은 종류의 균에 감염,사회 > 의료·건강
2640,경남 거제에서 3번째 콜레라 환자 나와,사회 > 의료·건강
1899,테니스 엘보 환자 10명 중 6명이 40 50대,뉴스 > IT/의학 > 건강
1467,암환자 세균감염 예방주사 부담 84만 4만원,뉴스 > 사회 > 사회일반
1243,거제서 세 번째 콜레라 환자,뉴스 > 사회 > 보건
2399,C형 간염 예방접종 없지만 치료율은 최대 90,사회 > 의료·건강





Cluster 7: Size 29


,title,category
2953,대전시 고도정수처리시설 준공식,사회 > 전국
372,여야 오늘 본회의서 추경 처리키로,정치
2836,정진석 야당 추경 처리 않으면 구조조정백남기 청문회 안돼,정치 > 정치일반
2314,여야 합의한 추경교육청에 2천억 누리과정 간접 지원,정치 > 국회·정당
1007,아무 일도 안한 정기국회 첫날,뉴스 > 정치 > 정치일반
1437,반칙왕 야당 부실 추경안 이정현추미애 체제 하루만에 충돌,뉴스 > 정치 > 정치일반
2169,국회 하루만에 정상화 추경 처리,정치 > 정치일반
2915,국회 교문위 추경 누리예산 야 단독 처리여 날치기,정치 > 국회·정당
2047,정기국회 전운 고조 노동검찰개혁 등 곳곳 지뢰,정치 > 국회·정당
906,찾아가는 뱅킹 서비스 은행 업무 대부분 현장에서 바로 처리,경제 > 재테크





Cluster 8: Size 69


,title,category
2330,9월 2일 인사,사회 > 인사
1121,인사인천환경공단,뉴스 > 사람속으로 > 인사일반
1336,인사아주대병원,뉴스 > 사람속으로 > 인사일반
1327,인사한국감정원,뉴스 > 사람속으로 > 인사일반
1338,인사한겨레신문,뉴스 > 사람속으로 > 인사일반
331,인사 황주호 한국원자력학회장 취임 외,사회 > 인물 > 인사
1333,인사서울여대,뉴스 > 사람속으로 > 인사일반
1915,인사국가보훈처,뉴스 > 사람속으로 > 인사일반
1331,인사강북삼성병원,뉴스 > 사람속으로 > 인사일반
1119,인사인천시,뉴스 > 사람속으로 > 인사일반


### Compare full cluster

In [31]:
cluster_data = []

for cluster_index in range(cluster_num):
    cluster_data.append(train[train['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i + 1), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

Cluster 1: Size 788


,title,category
2500,NH투자증권 세계적 투자은행 에버코어와 손잡아,경제 > 증권
910,은퇴 설계 늦었다고 생각할 때가 가장 빠른 시기,경제 > 재테크
19,국내 최대 규모 지상군 전시회,사회 > 국방
2444,법원 유우성 대북송금 기소는 공소권 남용검찰의 보복 기소 인정,사회 > 사회일반
2082,조달청 마피아 오명 조우회 순수 친목단체로 전환,사회 > 전국
1028,글로벌 특허괴물 2800여개 활개 툭하면 로열티 내라 경고장,뉴스 > 경제 > 경제일반
2470,이재오 우병우 놔두고 언론과 전면전하는 나쁜 권력,정치 > 정치일반
1768,한옥에 살다장명희요즘 한옥들 왜 판박이 모양일까,뉴스 > 문화 > 사설/칼럼
425,공항홍대공항 환승시간이 즐거워졌다,경제
199,멕시코 트럼프에게 이용당했다 부글부글,국제 > 미국ㆍ중남미





Cluster 2: Size 28


,title,category
524,의원 불체포특권 이번엔 손보나,정치 > 국회ㆍ정당
2128,박 대통령 5일 오전 시진핑 주석과 한중 정상회담,정치 > 청와대
773,온라인 당원 위세에 친문의원들도 꼼짝못하네,정치 > 국회ㆍ정당
2149,전직 경찰 총경 경찰 멱살 잡은 한선교 의원 고발할 것,정치 > 국회·정당
2831,우상호 김진태 동료 의원으로서 치욕스럽다,정치 > 정치BAR
1830,사설압수수색 받는 민정수석이 현직에 있는 게 정상인가,뉴스 > 정치 > 정치일반
2690,두테르테 초법적 살인에 맞선 강심장 의원,국제 > 아시아·태평양
365,한중한미 정상회담 잇달아 열릴 듯,정치 > 외교
2142,막말한다 인천 시의원 2명 술 취해 몸싸움,사회 > 사회일반
520,61개국 의원 참여 단체 서울서 북인권 규탄,정치 > 외교





Cluster 3: Size 49


...

,title,category
1078,김주원의 독서일기새로운 시작은 익숙한 것과의 결별,뉴스 > 문화 > 문화 일반
1143,한진해운발 물류 쇼크 운임 48 폭등,뉴스 > 경제 > 경제일반
2140,흉기난동 전 남편 목졸라 죽인 여성대법 정당방위 아냐,사회 > 사회일반
2571,김기춘이정현 등 증인 대거 불참120석 좁은 공간에 밀린 세월호 3차 청문회,사회 > 사회일반
2275,지동원이 날았다,스포츠 > 축구·해외리그
2930,사설 속으로 한겨레중앙일보 정운호 게이트 법조 비리 사설 비교해보기,사회 > 교육
1547,베이스볼 비키니선수는 노오력 지나 덕력의 시대로 가는데,뉴스 > 스포츠 > 스포츠일반
439,오승환 시즌 14세이브,스포츠ㆍ연예 > 스포츠 > 야구 > MLB > 코리안리거
289,한현우의 팝 컬처 위대한 코미디언들의 시대,사설ㆍ칼럼 > 내부칼럼 > 한현우의 팝 컬처
831,경주 신라왕경 복원 사업 2035년까지 1조4749억 투입,문화





Cluster 21: Size 29


,title,category
174,팔면봉 20대 첫 정기국회 개막날부터 국회의장 발언 정파성 논란으로 파행 외,사설ㆍ칼럼 > 내부칼럼 > 팔면봉
2836,정진석 야당 추경 처리 않으면 구조조정백남기 청문회 안돼,정치 > 정치일반
2165,새누리당의 강경투쟁 속에서도 막후접촉추경안 처리 도출,정치 > 국회·정당
2953,대전시 고도정수처리시설 준공식,사회 > 전국
906,찾아가는 뱅킹 서비스 은행 업무 대부분 현장에서 바로 처리,경제 > 재테크
1204,사설20대 첫 정기국회 파행 이젠 정의장까지 민생에 재 뿌리나,뉴스 > 정치 > 정치일반
2169,국회 하루만에 정상화 추경 처리,정치 > 정치일반
899,꽃도 정기구독 시대,경제 > 기업
249,100일간의 정기국회 대선 앞둬 곳곳 지뢰밭,정치 > 국회ㆍ정당
2515,추경 처리 9월 정기국회로 넘어간다,정치 > 국회·정당





Cluster 22: Size 69


,title,category
1158,경제계 인사대한건설협회 외,뉴스 > 경제 > 경제일반
1121,인사인천환경공단,뉴스 > 사람속으로 > 인사일반
1334,인사아주대,뉴스 > 사람속으로 > 인사일반
1118,인사한국철도기술연구원,뉴스 > 사람속으로 > 인사일반
1094,인사해양수산부,뉴스 > 사람속으로 > 인사일반
1331,인사강북삼성병원,뉴스 > 사람속으로 > 인사일반
1376,인사중소기업중앙회,뉴스 > 사람속으로 > 인사일반
1120,인사인천교통공사,뉴스 > 사람속으로 > 인사일반
1531,인사행정자치부,뉴스 > 사람속으로 > 인사일반
2071,9월 5일 인사,사회 > 인사
